In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataloader import create_data_loader
from model import TransformerModel, TransformerModel_ver2
from train import train_model,save_predictions, calculate_custom_accuracy



# Train

In [3]:
val_loader = create_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/val_df_l1p_2.csv',
    parquet_path='E:/DS/data/siam/bi/train_trans/train_l1p.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0,
    shuffle = False,
)

Preprocessing time series data...


100%|██████████████████████████████████████████████████████████████████████████| 44884/44884 [00:07<00:00, 6176.61it/s]


Preprocessed 4000 time series


In [4]:
# Create data loaders
train_loader = create_data_loader(
    csv_path='E:/DS/data/siam/bi/tgs/train_df_l1p_2.csv',
    parquet_path='E:/DS/data/siam/bi/train_trans/train_l1p.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=4
)






Preprocessing time series data...


100%|███████████████████████████████████████████████████████████████████████████| 44884/44884 [00:55<00:00, 806.41it/s]


Preprocessed 40884 time series


In [6]:
# Create model feats 3
model = TransformerModel_ver2(
    input_dim=3,
    d_model=256,
    nhead=4,
    num_encoder_layers=4,
    dim_feedforward=512,
    dropout=0.1,
    max_seq_len = 1024
)

In [5]:
# Create model
model = TransformerModel_ver2(
    input_dim=10,
    d_model=256,
    nhead=4,
    num_encoder_layers=4,
    dim_feedforward=512,
    dropout=0.1,
    max_seq_len = 1024
)

In [ ]:
# Train model
trained_model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=30,
    learning_rate=3e-4,
    device='cuda',
    binary_loss_weight=1.0,
    regression_loss_weight=1.0,
    pos_weight=1.0
)


Epoch 1/30


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 63/63 [00:07<00:00,  8.95it/s]


Train Loss: 0.9659, Binary Loss: 0.5102, Regression Loss: 0.4557
Val Loss: 0.5877, Binary Loss: 0.4476, Regression Loss: 0.1401, AUROC: 0.6710, MSE: 0.0446, Mean Accuracy: 0.6411
New best model saved with validation mean accuracy: 0.6411

Epoch 2/30


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 63/63 [00:06<00:00,  9.03it/s]


Train Loss: 0.5306, Binary Loss: 0.4140, Regression Loss: 0.1166
Val Loss: 0.4972, Binary Loss: 0.3960, Regression Loss: 0.1012, AUROC: 0.7778, MSE: 0.0322, Mean Accuracy: 0.6650
New best model saved with validation mean accuracy: 0.6650

Epoch 3/30


...

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 63/63 [00:06<00:00,  9.12it/s]


Train Loss: 0.4189, Binary Loss: 0.3392, Regression Loss: 0.0797
Val Loss: 0.4230, Binary Loss: 0.3376, Regression Loss: 0.0854, AUROC: 0.8598, MSE: 0.0272, Mean Accuracy: 0.6789

Epoch 6/30


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 63/63 [00:06<00:00,  9.07it/s]


Train Loss: 0.4077, Binary Loss: 0.3311, Regression Loss: 0.0766
Val Loss: 0.4121, Binary Loss: 0.3335, Regression Loss: 0.0786, AUROC: 0.8656, MSE: 0.0250, Mean Accuracy: 0.7164
New best model saved with validation mean accuracy: 0.7164

Epoch 7/30


Training:  41%|██████▍         | 259/639 [01:50<02:03,  3.08it/s, loss=0.374, binary_loss=0.34, regression_loss=0.0343]

In [7]:
import torch

In [8]:
torch.save(trained_model.state_dict(), 'E:/DS/data/siam/weights/ts_3f_l1p_2.pth')

In [9]:
test_loader  = create_data_loader(
    csv_path=None,
    parquet_path='E:/DS/data/siam/bi/test_trans/test_l1p_adv.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)

Preprocessing time series data...


100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1929.54it/s]


Preprocessed 500 time series


In [10]:
save_predictions(trained_model,test_loader,
                'E:/DS/data/siam/bi/inference/trans_enc_v1/inf_l1p_ts3_v2.csv',
                'cuda',
                binary_threshold=0.5)

Running inference: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.72it/s]

Predictions saved to E:/DS/data/siam/bi/inference/trans_enc_v1/inf_l1p_ts10_v1.csv


# Prep data for public

In [11]:
import numpy as np
import pandas as pd

In [12]:
ch=pd.read_csv('E:/DS/data/siam/bi/inference/trans_enc_v1/inf_l1p_ts3_v2.csv')

In [13]:
keep_cols = ['file_name',
       'binary_Некачественное ГДИС', 
       'binary_Влияние ствола скважины', 
       'binary_Радиальный режим',
       'binary_Линейный режим', 
       'binary_Билинейный режим', 
       'binary_Сферический режим', 
       'binary_Граница постоянного давления',
       'binary_Граница непроницаемый разлом',
       'regression_Влияние ствола скважины_details',
       'regression_Радиальный режим_details',
       'regression_Линейный режим_details',
       'regression_Билинейный режим_details',
       'regression_Сферический режим_details',
       'regression_Граница постоянного давления_details',
       'regression_Граница непроницаемый разлом_details']

In [14]:
import pickle

def load_scaler(file_path):
    """
    Load an sklearn scaler from a pickle file.

    Args:
        file_path (str): Path to the scaler file.

    Returns:
        scaler: The loaded scaler object.
    """
    try:
        with open(file_path, 'rb') as file:
            scaler = pickle.load(file)
        print(f"Scaler loaded successfully from {file_path}")
        return scaler
    except Exception as e:
        print(f"Error loading scaler: {e}")
        return None

In [15]:
def apply_trans_reg(ch, path_to_scalers='E:/DS/data/siam/scalers/tg_scalers_r/tg_r_'):
    for i, col in enumerate(['regression_Влияние ствола скважины_details',
           'regression_Радиальный режим_details',
           'regression_Линейный режим_details',
           'regression_Билинейный режим_details',
           'regression_Сферический режим_details',
           'regression_Граница постоянного давления_details',
           'regression_Граница непроницаемый разлом_details']):
        print(col)
        curr_path = path_to_scalers+f'{i}.pkl'
        r_scaler = load_scaler(curr_path)
        
        data = ch[[col]]
        #print(col, data.shape)
        data = r_scaler.inverse_transform(data)
        #print(col, data.shape)
        ch[col]=data
    return ch

In [17]:
ch = apply_trans_reg(ch, path_to_scalers='E:/DS/data/siam/scalers/tg_scalers_r/tg_r_')

regression_Влияние ствола скважины_details
Scaler loaded successfully from E:/DS/data/siam/scalers/tg_scalers_r/tg_r_0.pkl
regression_Радиальный режим_details
Scaler loaded successfully from E:/DS/data/siam/scalers/tg_scalers_r/tg_r_1.pkl
regression_Линейный режим_details
Scaler loaded successfully from E:/DS/data/siam/scalers/tg_scalers_r/tg_r_2.pkl
regression_Билинейный режим_details
Scaler loaded successfully from E:/DS/data/siam/scalers/tg_scalers_r/tg_r_3.pkl
regression_Сферический режим_details
Scaler loaded successfully from E:/DS/data/siam/scalers/tg_scalers_r/tg_r_4.pkl
regression_Граница постоянного давления_details
Scaler loaded successfully from E:/DS/data/siam/scalers/tg_scalers_r/tg_r_5.pkl
regression_Граница непроницаемый разлом_details
Scaler loaded successfully from E:/DS/data/siam/scalers/tg_scalers_r/tg_r_6.pkl


In [15]:
ch[columns_r_2] = np.expm1(ch[columns_r_2])

In [16]:
ch = ch[keep_cols]

In [17]:
df = pd.read_csv('E:/DS/data/siam/bi/markup_train.csv')

In [18]:
df.describe()

,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
count,45141.000000,45141.000000,45141.000000,45141.000000,45141.000000,45141.000000,45141.000000,45141.000000,35558.000000,25855.000000,12211.000000,12086.000000,5764.000000,4874.000000,4265.000000
mean,0.160984,0.787710,0.572761,0.270508,0.267739,0.127689,0.107973,0.094482,1.719234,1.140486,0.605509,0.934330,1.292040,144.019966,160.928583
std,0.367521,0.408934,0.494683,0.444227,0.442786,0.333747,0.310349,0.292501,1.537100,1.155191,1.012616,1.166539,0.982978,782.593841,339.370082
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.843521,-3.707246,-4.109357,-3.332898,-5.164659,0.006653,0.010753
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.914535,0.504870,0.028459,0.327374,0.743594,18.335495,29.301152
50%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.589366,0.959171,0.516372,0.720093,1.296814,55.182021,75.822331
75%,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.256645,1.422883,1.024682,1.130444,1.808614,134.488126,182.395620
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,31.777765,6.408605,29.027806,29.129381,6.631359,51035.492870,11595.502940


In [18]:
ch.describe()

,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
count,500.00000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.14600,0.802000,0.574000,0.208000,0.194000,0.138000,0.168000,0.170000,1.514535,0.830428,0.557185,0.699741,1.112256,163.048661,124.206743
std,0.35346,0.398891,0.494989,0.406283,0.395825,0.345245,0.374241,0.376009,1.490684,0.970351,1.073124,0.939794,0.982531,356.043289,245.323979
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.986945,-0.970029,-0.996172,-0.979064,-0.967487,-0.221184,0.221630
25%,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.675532,0.275074,-0.142879,0.187088,0.451885,20.222968,21.503734
50%,0.00000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.461642,0.756100,0.401800,0.635850,1.088484,58.536489,63.128304
75%,0.00000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.082922,1.264266,1.014480,1.063428,1.763843,149.523001,143.859237
max,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.099486,4.855204,5.677583,5.567347,5.377112,3874.457503,4403.262055


In [14]:
sb = pd.read_csv('E:/DS/data/siam/bi/inference/random/submit1.csv')
columns_b=['Некачественное ГДИС',
       'Влияние ствола скважины', 'Радиальный режим', 'Линейный режим',
       'Билинейный режим', 'Сферический режим', 'Граница постоянного давления',
       'Граница непроницаемый разлом', ]
columns_r=['Влияние ствола скважины_details',
       'Радиальный режим_details', 'Линейный режим_details',
       'Билинейный режим_details', 'Сферический режим_details',
       'Граница постоянного давления_details',
       'Граница непроницаемый разлом_details']

columns_b_2 = ['binary_Некачественное ГДИС', 
       'binary_Влияние ствола скважины', 
       'binary_Радиальный режим',
       'binary_Линейный режим', 
       'binary_Билинейный режим', 
       'binary_Сферический режим', 
       'binary_Граница постоянного давления',
       'binary_Граница непроницаемый разлом',]
columns_r_2 =['regression_Влияние ствола скважины_details',
       'regression_Радиальный режим_details',
       'regression_Линейный режим_details',
       'regression_Билинейный режим_details',
       'regression_Сферический режим_details',
       'regression_Граница постоянного давления_details',
       'regression_Граница непроницаемый разлом_details']

In [17]:
ch.columns  = ['file_name']+columns_b+columns_r

In [19]:
ch = sb[['file_name']].merge(ch)

In [20]:
ch.to_csv('E:/DS/data/siam/bi/inference/trans_enc_v1/inf_l1p_ts3_v2_tr.csv',index=False)

# blend